In [1]:
import joblib

import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

from mlflow.tracking import MlflowClient

In [6]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow_model.db")

mlflow.set_experiment("wine_quality_clf")

2022/09/12 14:24:21 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/09/12 14:24:21 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

INFO: 'wine_quality_clf' does not exist. Creating a new experiment


In [7]:
def read_dataframe():
    redwine = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
    redwine['type'] = 'red'

    whitewine = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')
    whitewine['type'] = 'white'

    df = pd.concat([redwine, whitewine], axis = 0).reset_index(drop=True)
    
    numeric_columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']

    df['quality'] =  np.where(df.quality.isin([3,4]), '3-4', np.where(df.quality.isin([8,9]), '8-9', df.quality))
    df['quality'] =  df.quality.astype('object')
    
    return df, numeric_columns


def train_model(df, numeric_columns):
    X_train, X_test, y_train, y_test = train_test_split(df.drop('quality', axis=1), df.quality,
                                                    stratify=df.quality, 
                                                    test_size=0.3,
                                                    random_state=123)


    pipe = Pipeline([
        ('column_transformer', ColumnTransformer([
            ('one-hot', OneHotEncoder(handle_unknown='ignore'), ['type']),
            ('scaler', StandardScaler(), numeric_columns)], remainder='drop')),
        ('model', OneVsRestClassifier(xgb.XGBClassifier(random_state= 123, eval_metric='logloss')))])
            
    return X_train, X_test, y_train, y_test, pipe

In [8]:
df, numeric_columns = read_dataframe()

X_train, X_test, y_train, y_test, pipe = train_model(df, numeric_columns)

In [9]:
with mlflow.start_run():
    params = dict(random_state= 123, eval_metric='logloss')
    mlflow.log_params(params)

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test) 

    accuracy = accuracy_score(y_pred, y_test)
    print('Model Accuracy :', accuracy)
    mlflow.log_metric('accuracy', accuracy)

    mlflow.sklearn.log_model(pipe, artifact_path="model")

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


Model Accuracy : 0.6594871794871795


INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [11]:
client = MlflowClient(tracking_uri="sqlite:///mlflow_model.db")

client.list_experiments()

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='wine_quality_clf', tags={}>]

In [13]:
runs = client.search_runs(
    experiment_ids='1'
)

for run in runs:
    print(f"run id: {run.info.run_id}, accuracy: {run.data.metrics['accuracy']:.4f}")

run id: 0c97a1e2b4a2472f825f4c29c5711b56, accuracy: 0.6595


In [16]:
run_id = '0c97a1e2b4a2472f825f4c29c5711b56'
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="wine_quality_clf")

INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Successfully registered model 'wine_quality_clf'.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2022/09/12 14:31:53 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: wine_quality_clf, version 1
Created version '1' of model 'wine_quality_clf'.


<ModelVersion: creation_timestamp=1662982312990, current_stage='None', description=None, last_updated_timestamp=1662982312990, name='wine_quality_clf', run_id='0c97a1e2b4a2472f825f4c29c5711b56', run_link=None, source='./mlruns/1/0c97a1e2b4a2472f825f4c29c5711b56/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [18]:
model_version = 1
model_name = "wine_quality_clf"
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1662982312990, current_stage='Staging', description=None, last_updated_timestamp=1662982395405, name='wine_quality_clf', run_id='0c97a1e2b4a2472f825f4c29c5711b56', run_link=None, source='./mlruns/1/0c97a1e2b4a2472f825f4c29c5711b56/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [19]:

latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Staging


In [20]:
model_version = 1
model_name = "wine_quality_clf"
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1662982312990, current_stage='Production', description=None, last_updated_timestamp=1662982450040, name='wine_quality_clf', run_id='0c97a1e2b4a2472f825f4c29c5711b56', run_link=None, source='./mlruns/1/0c97a1e2b4a2472f825f4c29c5711b56/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [22]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1662982312990, current_stage='Production', description=None, last_updated_timestamp=1662982472604, name='wine_quality_clf', run_id='0c97a1e2b4a2472f825f4c29c5711b56', run_link=None, source='./mlruns/1/0c97a1e2b4a2472f825f4c29c5711b56/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>